In [1]:
!pip install torch transformers datasets sentencepiece sacrebleu accelerate

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import torch
from datasets import Dataset
from transformers import (
    M2M100ForConditionalGeneration,
    M2M100Tokenizer,
    DataCollatorForSeq2Seq,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
    AutoModelForSeq2SeqLM
)

In [3]:
print(f"Device: {torch.cuda.get_device_name(0)}")
print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

MODEL_NAME = "facebook/m2m100_418M"

tokenizer = M2M100Tokenizer.from_pretrained(MODEL_NAME)
model = M2M100ForConditionalGeneration.from_pretrained(MODEL_NAME).cuda()

Device: NVIDIA H200
VRAM: 150.02 GB


In [4]:
for param in model.parameters():
    param.requires_grad = True

print("Đã mở băng toàn bộ Model. Sẵn sàng huấn luyện chuyên sâu.")

Đã mở băng toàn bộ Model. Sẵn sàng huấn luyện chuyên sâu.


In [5]:
# %%
DATA_DIR = "/home/admin/dataset"

def load_parallel(src_file, tgt_file):
    with open(src_file, encoding="utf-8") as f:
        src = [l.strip() for l in f]
    with open(tgt_file, encoding="utf-8") as f:
        tgt = [l.strip() for l in f]
    assert len(src) == len(tgt)
    return Dataset.from_dict({
        "src_text": src,
        "tgt_text": tgt
    })


In [6]:

train_dataset = load_parallel(
    f"{DATA_DIR}/train2022.zh",  
    f"{DATA_DIR}/train2022.vi"    
)

dev_dataset = load_parallel(
    f"{DATA_DIR}/dev2022.zh.txt",
    f"{DATA_DIR}/dev2022.vi.txt"
)

print(f"Train: {len(train_dataset)} | Dev: {len(dev_dataset)}")


Train: 300348 | Dev: 1000


In [7]:
MAX_LEN = 256

def preprocess(batch):
    tokenizer.src_lang = "zh"
    tokenizer.tgt_lang = "vi"
    inputs = tokenizer(batch["src_text"], truncation=True, max_length=MAX_LEN)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(batch["tgt_text"], truncation=True, max_length=MAX_LEN)
    inputs["labels"] = labels["input_ids"]
    return inputs



In [8]:

train_dataset = train_dataset.map(
    preprocess,
    batched=True,
    remove_columns=train_dataset.column_names,
    num_proc=8
)

dev_dataset = dev_dataset.map(
    preprocess,
    batched=True,
    remove_columns=dev_dataset.column_names,
    num_proc=8
)


Map (num_proc=8):   0%|          | 0/300348 [00:00<?, ? examples/s]

/home/admin/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/admin/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/admin/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers

Map (num_proc=8):   0%|          | 0/1000 [00:00<?, ? examples/s]

/home/admin/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/admin/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/admin/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers

In [9]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True
)

In [15]:

training_args = TrainingArguments(
    output_dir="/home/admin/checkpoint",
    eval_strategy="steps",
    save_strategy="steps",
    eval_steps=500,
    save_steps=500,
    logging_steps=100,

    per_device_train_batch_size=128,
    per_device_eval_batch_size=64,

    bf16=True,
    fp16=False,

    learning_rate=5e-5,
    num_train_epochs=10,
    warmup_steps=1000,
    save_total_limit=2,

    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="none"
)


In [16]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)


/tmp/ipykernel_15923/1190918925.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [17]:
trainer.train(resume_from_checkpoint=True)


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


Step,Training Loss,Validation Loss
3000,2.007600,1.527786
3500,1.922700,1.519608
4000,1.897600,1.505428
4500,1.889400,1.495252
5000,1.780800,1.497468
5500,1.788200,1.492633
6000,1.777800,1.481292
6500,1.772400,1.471444
7000,1.772600,1.460001
7500,1.651100,1.475682


/home/admin/.local/lib/python3.10/site-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=12000, training_loss=1.3592805417378744, metrics={'train_runtime': 1556.9176, 'train_samples_per_second': 1929.119, 'train_steps_per_second': 15.075, 'total_flos': 2.789931480589271e+17, 'train_loss': 1.3592805417378744, 'epoch': 5.112910097997443})

In [18]:
trainer.save_model("/home/admin/checkpoint/best_zh_vi")
tokenizer.save_pretrained("/home/admin/checkpoint/best_zh_vi")

('/home/admin/checkpoint/best_zh_vi/tokenizer_config.json',
 '/home/admin/checkpoint/best_zh_vi/special_tokens_map.json',
 '/home/admin/checkpoint/best_zh_vi/vocab.json',
 '/home/admin/checkpoint/best_zh_vi/sentencepiece.bpe.model',
 '/home/admin/checkpoint/best_zh_vi/added_tokens.json')

In [19]:

from sacrebleu import corpus_bleu


In [20]:
def translate_zh_vi(src_file):
    outputs = []
    model.eval()

    tokenizer.src_lang = "zh"
    tokenizer.tgt_lang = "vi"

    with open(src_file, encoding="utf-8") as f:
        lines = [l.strip() for l in f if l.strip()]

    print(f"Đang dịch {len(lines)} câu ZH → VI")

    for line in lines:
        inputs = tokenizer(
            line,
            return_tensors="pt",
            truncation=True,
            max_length=256
        ).to("cuda")

        with torch.no_grad():
            gen = model.generate(
                **inputs,
                forced_bos_token_id=tokenizer.get_lang_id("vi"),  
                num_beams=5,
                max_length=256
            )

        outputs.append(
            tokenizer.decode(gen[0], skip_special_tokens=True)
        )

    return outputs


In [21]:
preds = translate_zh_vi(f"{DATA_DIR}/test.vi-zh.2022.zh")

# Đọc file reference tương ứng
with open(f"{DATA_DIR}/test.vi-zh.2022.vi", encoding="utf-8") as f:
    refs = [f.read().splitlines()]

# Tính BLEU
bleu = corpus_bleu(preds, refs, tokenize='intl')
print("\n" + "="*30)
print(f"KẾT QUẢ ZH-VI TRÊN H200")
print(f"SacreBLEU Score: {bleu.score:.2f}")
print("="*30)

Đang dịch 1000 câu ZH → VI

KẾT QUẢ ZH-VI TRÊN H200
SacreBLEU Score: 36.84


In [22]:

import random

N = 5  # số câu muốn xem ngẫu nhiên

# Đọc source ZH
with open(f"{DATA_DIR}/test.vi-zh.2022.zh", encoding="utf-8") as f:
    src_lines = [l.strip() for l in f if l.strip()]

# Đọc reference VI
with open(f"{DATA_DIR}/test.vi-zh.2022.vi", encoding="utf-8") as f:
    ref_lines = [l.strip() for l in f if l.strip()]

assert len(src_lines) == len(ref_lines)

model.eval()
tokenizer.src_lang = "zh"
tokenizer.tgt_lang = "vi"

idxs = random.sample(range(len(src_lines)), N)

for i, idx in enumerate(idxs):
    src = src_lines[idx]
    ref = ref_lines[idx]

    inputs = tokenizer(
        src,
        return_tensors="pt",
        truncation=True,
        max_length=256
    ).to("cuda")

    with torch.no_grad():
        gen = model.generate(
            **inputs,
            forced_bos_token_id=tokenizer.get_lang_id("vi"),
            num_beams=5,
            max_length=256
        )

    pred = tokenizer.decode(gen[0], skip_special_tokens=True)

    print(f"\n--- CÂU NGẪU NHIÊN {i+1} (idx={idx}) ---")
    print("ZH (SOURCE):")
    print(src)
    print("\nVI (MODEL):")
    print(pred)
    print("\nVI (REAL):")
    print(ref)
    print("=" * 60)



--- CÂU NGẪU NHIÊN 1 (idx=654) ---
ZH (SOURCE):
公安力量帮助胡志明市居民购买粮食食品越南政府总理范明政在会上发表讲话时强调，既然决定牺牲经济利益，实施社交距离措施，就要成功控制疫情，要尽可能早和有效击退疫情。

VI (MODEL):
Trong bài phát biểu tại hội nghị, lực lượng công an hỗ trợ người dân mua lương thực và lương thực Thành phố Hồ Chí Minh, Thủ tướng Phạm Minh Chính khẳng định quyết định hy sinh lợi ích kinh tế, thực hiện các biện pháp cách mạng xã hội thì phải kiểm soát thành công dịch bệnh, sớm và hiệu quả nhất có thể.

VI (REAL):
Phát biểu tại hội nghị, người đứng đầu Chính phủ nhấn mạnh tinh thần chung: Việt Nam đã hy sinh về kinh tế - xã hội để thực hiện giãn cách thì phải đạt kết quả chống dịch thành công, phải ngăn chặn, đẩy lùi được dịch bệnh sớm, nhanh, hiệu quả.

--- CÂU NGẪU NHIÊN 2 (idx=114) ---
ZH (SOURCE):
现在落实第128号决议，我们将审视、颁布新规定，确保符合决议目的、要求及卫生部相关防疫规定。

VI (MODEL):
Bây giờ thực hiện Nghị quyết 128 chúng ta sẽ rà soát, ban hành các quy định mới, bảo đảm phù hợp với mục đích, yêu cầu của Nghị quyết cũng như các quy định về phòng chống dịch bệnh của Bộ

In [24]:
import shutil
import os

src_dir = "/home/admin/checkpoint/best_zh_vi"
zip_path = "/home/admin/checkpoint/best_zh_vi"

shutil.make_archive(zip_path, 'zip', src_dir)

print("Done:", zip_path + ".zip")

Done: /home/admin/checkpoint/best_zh_vi.zip
